In [1]:
import pyvista as pv
pv.global_theme.jupyter_backend = "none"

In [2]:
from OCC.Core.Bnd import Bnd_Box
from OCC.Core.BRepBndLib import brepbndlib

def compute_bounding_box(shape):
    bbox = Bnd_Box()
    brepbndlib.Add(shape, bbox)
    xmin, ymin, zmin, xmax, ymax, zmax = bbox.Get()

    length = xmax - xmin  # aligned along X
    height = ymax - ymin  # aligned along Y
    width  = zmax - zmin  # aligned along Z

    return {
        "length": length,
        "height": height,
        "width": width,
        "xmin": xmin, "xmax": xmax,
        "ymin": ymin, "ymax": ymax,
        "zmin": zmin, "zmax": zmax
    }


In [3]:
from OCC.Core.STEPControl import STEPControl_Reader
from OCC.Core.IFSelect import IFSelect_RetDone
from OCC.Core.Bnd import Bnd_Box
from OCC.Core.BRepBndLib import brepbndlib

pv.set_jupyter_backend("none")

# Load the STEP file
reader = STEPControl_Reader()
status = reader.ReadFile("0444-1.step")
reader.TransferRoots()
shape = reader.OneShape()

# Compute bounding box and print dimensions
if not shape.IsNull():
    bbox = Bnd_Box()
    brepbndlib.Add(shape, bbox)
    xmin, ymin, zmin, xmax, ymax, zmax = bbox.Get()
    
    length = xmax - xmin
    height = ymax - ymin
    width  = zmax - zmin  # Assuming beam is aligned with X axis

    print(f"Beam bounding box (mm):")
    print(f"  Length: {length:.2f}")
    print(f"  Height: {height:.2f}")
    print(f"  Width : {width:.2f}")
else:
    print("Error: STEP shape is null.")


Beam bounding box (mm):
  Length: 203.60
  Height: 203.20
  Width : 2280.00


In [4]:
import numpy as np
import pyvista as pv
from OCC.Core.BRep import BRep_Tool
from OCC.Extend.TopologyUtils import TopologyExplorer
from OCC.Core.BRepMesh import BRepMesh_IncrementalMesh
from sklearn.decomposition import PCA

# --- Extract vertices for PCA ---
def get_shape_vertices(shape):
    topo = TopologyExplorer(shape)
    points = []
    for vertex in topo.vertices():
        p = BRep_Tool.Pnt(vertex)
        points.append([p.X(), p.Y(), p.Z()])
    return np.array(points)

# --- Run PCA and get axes ---
def get_pca_axes(points):
    pca = PCA(n_components=3)
    pca.fit(points)
    return pca.mean_, pca.components_

# --- Convert shape to mesh for PyVista ---
def shape_to_pyvista_mesh(shape):
    from OCC.Core.BRep import BRep_Tool
    from OCC.Extend.TopologyUtils import TopologyExplorer
    vertices = []
    faces = []

    BRepMesh_IncrementalMesh(shape, 0.5)
    topo = TopologyExplorer(shape)

    for face in topo.faces():
        triangulation = BRep_Tool.Triangulation(face, face.Location())
        if triangulation is None:
            continue

        start_idx = len(vertices)
        for i in range(1, triangulation.NbNodes() + 1):
            p = triangulation.Node(i)
            vertices.append([p.X(), p.Y(), p.Z()])

        for i in range(1, triangulation.NbTriangles() + 1):
            t = triangulation.Triangle(i)
            n1, n2, n3 = t.Get()
            faces.append([3, start_idx + n1 - 1, start_idx + n2 - 1, start_idx + n3 - 1])

    return pv.PolyData(np.array(vertices), np.array(faces))

# --- Load and visualize ---
points = get_shape_vertices(shape)
center, axes = get_pca_axes(points)
mesh = shape_to_pyvista_mesh(shape)

# Draw PCA axes (length = 1000 units)
lines = []
colors = [(1, 0, 0), (0, 1, 0), (0, 0, 1)]  # R, G, B
labels = ['X', 'Y', 'Z']
for i, axis in enumerate(axes):
    start = center
    end = center + axis * 1000
    line = pv.Line(start, end)
    line["scalars"] = [i, i]  # dummy scalar
    lines.append(line)

plotter = pv.Plotter()
plotter.add_mesh(mesh, show_edges=True, color='lightgray')
for i, line in enumerate(lines):
    plotter.add_mesh(line, color=colors[i], line_width=4)
    plotter.add_point_labels([center + axes[i] * 1000], [labels[i]], font_size=12)


plotter = pv.Plotter()
plotter.add_mesh(mesh, show_edges=True, color='lightgray')

for i, line in enumerate(lines):
    plotter.add_mesh(line, color=colors[i], line_width=4)
    plotter.add_point_labels([center + axes[i] * 1000], [labels[i]], font_size=12)

plotter.screenshot("beam_with_pca_axes.png")

bounds = mesh.bounds  # (xmin, xmax, ymin, ymax, zmin, zmax)
xmin, xmax, ymin, ymax, zmin, zmax = bounds

length = xmax - xmin
height = ymax - ymin
width  = zmax - zmin
center = mesh.center

print("Unaligned Mesh")
print("✅ Mesh bounding box (mm):")
print(f"  Length (X): {length:.2f}")
print(f"  Height (Y): {height:.2f}")
print(f"  Width  (Z): {width:.2f}")
print(f"  Mesh center: {center}")



Unaligned Mesh
✅ Mesh bounding box (mm):
  Length (X): 203.60
  Height (Y): 203.20
  Width  (Z): 2280.00
  Mesh center: (0.0, 7.105427357601002e-15, 1140.0)


In [5]:
from sklearn.decomposition import PCA

points = get_shape_vertices(shape)  # shape = your TopoDS_Shape
pca = PCA(n_components=3).fit(points)


In [6]:
def build_precise_alignment_matrix(pca):
    """
    Build a strictly orthonormal rotation matrix from PCA axes.
    Ensures: axes[2] = cross(axes[0], axes[1])
    """
    axes = pca.components_

    # Step 1: Assume PCA axis 0 is length (aligned to X)
    x_axis = axes[0]

    # Step 2: Pick orthogonal Y (even if noisy)
    raw_y = axes[1]
    y_axis = raw_y - np.dot(raw_y, x_axis) * x_axis
    y_axis /= np.linalg.norm(y_axis)

    # Step 3: Z = X × Y
    z_axis = np.cross(x_axis, y_axis)

    return np.column_stack([x_axis, y_axis, z_axis])

In [7]:
def align_mesh_with_custom_matrix(mesh, center, rotation_matrix):
    """
    Apply a custom rotation matrix and center shift to a PyVista mesh.
    
    Parameters:
        mesh: pyvista.PolyData — the original mesh
        center: np.array(3,) — the point to translate to origin before rotation
        rotation_matrix: np.array(3x3) — axes as columns, to align mesh to global frame
    
    Returns:
        aligned_mesh: pyvista.PolyData — transformed mesh
    """
    points = mesh.points - center  # center it
    aligned_points = np.dot(points, rotation_matrix)  # rotate
    aligned_mesh = mesh.copy()
    aligned_mesh.points = aligned_points
    return aligned_mesh


In [8]:
import pyvista as pv

precise_rot = build_precise_alignment_matrix(pca)
aligned_mesh = align_mesh_with_custom_matrix(mesh, center, precise_rot)

bounds = aligned_mesh.bounds
print("✅ Aligned mesh dimensions (mm):")
print(f"  Length (X): {bounds[1] - bounds[0]:.2f}")
print(f"  Height (Y): {bounds[3] - bounds[2]:.2f}")
print(f"  Width  (Z): {bounds[5] - bounds[4]:.2f}")

# Create bounding box as a wireframe cube
bbox_mesh = aligned_mesh.bounding_box()

# Visualize
plotter = pv.Plotter()
plotter.add_mesh(aligned_mesh, show_edges=True, color='lightgray', opacity=1.0)
plotter.add_mesh(bbox_mesh, style='wireframe', color='red', line_width=2)

# Optional: add global axes
plotter.add_axes(line_width=2)

# Save screenshot or show (depending on environment)
plotter.screenshot("aligned_mesh_with_bbox.png")
# or try: plotter.show(jupyter_backend="none")  # in script/terminal, not JupyterLab


✅ Aligned mesh dimensions (mm):
  Length (X): 2281.37
  Height (Y): 203.20
  Width  (Z): 219.52


pyvista_ndarray([[[255, 255, 255],
                  [255, 255, 255],
                  [255, 255, 255],
                  ...,
                  [255, 255, 255],
                  [255, 255, 255],
                  [255, 255, 255]],

                 [[255, 255, 255],
                  [255, 255, 255],
                  [255, 255, 255],
                  ...,
                  [255, 255, 255],
                  [255, 255, 255],
                  [255, 255, 255]],

                 [[255, 255, 255],
                  [255, 255, 255],
                  [255, 255, 255],
                  ...,
                  [255, 255, 255],
                  [255, 255, 255],
                  [255, 255, 255]],

                 ...,

                 [[255, 255, 255],
                  [255, 255, 255],
                  [255, 255, 255],
                  ...,
                  [255, 255, 255],
                  [255, 255, 255],
                  [255, 255, 255]],

                 [[255, 255, 255],
  

In [9]:
# Dot product between axes
print("X ⋅ Y:", np.dot(pca.components_[0], pca.components_[1]))
print("X ⋅ Z:", np.dot(pca.components_[0], pca.components_[2]))
print("Y ⋅ Z:", np.dot(pca.components_[1], pca.components_[2]))
print("X norm:", np.linalg.norm(pca.components_[0]))
print("Y norm:", np.linalg.norm(pca.components_[1]))
print("Z norm:", np.linalg.norm(pca.components_[2]))

X ⋅ Y: -3.2590264525655566e-16
X ⋅ Z: -8.673617379884035e-19
Y ⋅ Z: -3.0117621044360546e-18
X norm: 0.9999999999999999
Y norm: 0.9999999999999999
Z norm: 1.0


In [10]:
points_pca = pca.transform(points)


In [11]:
points = mesh.points  # Get points from the PyVista mesh
pca = PCA(n_components=3).fit(points)
points_pca = pca.transform(points)

pca_mesh = mesh.copy()
pca_mesh.points = points_pca

# Bounding box and visual check
bounds = pca_mesh.bounds
print("Bounding box in PCA space:")
print(f"  X: {bounds[0]:.2f} to {bounds[1]:.2f} ({bounds[1] - bounds[0]:.2f})")
print(f"  Y: {bounds[2]:.2f} to {bounds[3]:.2f} ({bounds[3] - bounds[2]:.2f})")
print(f"  Z: {bounds[4]:.2f} to {bounds[5]:.2f} ({bounds[5] - bounds[4]:.2f})")

# Optional visualization
plotter = pv.Plotter()
plotter.add_mesh(pca_mesh, show_edges=True, color='lightgray')
plotter.add_mesh(pca_mesh.bounding_box(), style='wireframe', color='red', line_width=2)
plotter.add_axes()
plotter.screenshot("pca_mesh_preview.png")


Bounding box in PCA space:
  X: -1066.04 to 1216.43 (2282.47)
  Y: -101.60 to 101.60 (203.20)
  Z: -124.32 to 109.09 (233.40)


pyvista_ndarray([[[255, 255, 255],
                  [255, 255, 255],
                  [255, 255, 255],
                  ...,
                  [255, 255, 255],
                  [255, 255, 255],
                  [255, 255, 255]],

                 [[255, 255, 255],
                  [255, 255, 255],
                  [255, 255, 255],
                  ...,
                  [255, 255, 255],
                  [255, 255, 255],
                  [255, 255, 255]],

                 [[255, 255, 255],
                  [255, 255, 255],
                  [255, 255, 255],
                  ...,
                  [255, 255, 255],
                  [255, 255, 255],
                  [255, 255, 255]],

                 ...,

                 [[255, 255, 255],
                  [255, 255, 255],
                  [255, 255, 255],
                  ...,
                  [255, 255, 255],
                  [255, 255, 255],
                  [255, 255, 255]],

                 [[255, 255, 255],
  

In [12]:
def get_ordered_pca_rotation(pca, points):
    """
    Orders PCA components so:
    - Longest spread axis becomes X (length)
    - Next becomes Y (height)
    - Third becomes Z (width)
    """
    pca_coords = pca.transform(points)
    spreads = np.ptp(pca_coords, axis=0)

    # Get axis indices by decreasing spread
    sorted_indices = np.argsort(spreads)[::-1]
    x_axis = pca.components_[sorted_indices[0]]
    y_axis = pca.components_[sorted_indices[1]]

    # Make Y orthogonal to X
    y_axis = y_axis - np.dot(y_axis, x_axis) * x_axis
    y_axis /= np.linalg.norm(y_axis)

    # Z = X × Y
    z_axis = np.cross(x_axis, y_axis)

    rotation = np.column_stack([x_axis, y_axis, z_axis])  # shape (3, 3)
    return rotation


In [20]:
# PCA Detailed
points = mesh.points
pca = PCA(n_components=3).fit(points)
rotation = get_ordered_pca_rotation(pca, points)
center = pca.mean_

def apply_rotation_to_mesh(mesh, center, rotation_matrix):
    centered = mesh.points - center
    rotated = np.dot(centered, rotation_matrix)
    aligned = mesh.copy()
    aligned.points = rotated
    return aligned

aligned_mesh = apply_rotation_to_mesh(mesh, center, rotation)

# Check results
bounds = aligned_mesh.bounds
print("Aligned mesh bounding box:")
print(f"  X: {bounds[0]:.2f} to {bounds[1]:.2f} ({bounds[1]-bounds[0]:.2f})")
print(f"  Y: {bounds[2]:.2f} to {bounds[3]:.2f} ({bounds[3]-bounds[2]:.2f})")
print(f"  Z: {bounds[4]:.2f} to {bounds[5]:.2f} ({bounds[5]-bounds[4]:.2f})")

# Preview
plotter = pv.Plotter()
plotter.add_mesh(aligned_mesh, show_edges=True, color='lightgray')
plotter.add_mesh(aligned_mesh.bounding_box(), style='wireframe', color='red', line_width=2)
plotter.add_axes()
plotter.screenshot("final_aligned_mesh.png")


Aligned mesh bounding box:
  X: -1066.04 to 1216.43 (2282.47)
  Y: -124.32 to 109.09 (233.40)
  Z: -101.60 to 101.60 (203.20)


pyvista_ndarray([[[255, 255, 255],
                  [255, 255, 255],
                  [255, 255, 255],
                  ...,
                  [255, 255, 255],
                  [255, 255, 255],
                  [255, 255, 255]],

                 [[255, 255, 255],
                  [255, 255, 255],
                  [255, 255, 255],
                  ...,
                  [255, 255, 255],
                  [255, 255, 255],
                  [255, 255, 255]],

                 [[255, 255, 255],
                  [255, 255, 255],
                  [255, 255, 255],
                  ...,
                  [255, 255, 255],
                  [255, 255, 255],
                  [255, 255, 255]],

                 ...,

                 [[255, 255, 255],
                  [255, 255, 255],
                  [255, 255, 255],
                  ...,
                  [255, 255, 255],
                  [255, 255, 255],
                  [255, 255, 255]],

                 [[255, 255, 255],
  

In [21]:


import numpy as np
from sklearn.linear_model import RANSACRegressor
from sklearn.decomposition import PCA

def fit_ransac_line(points):
    """
    Fit a 3D line to the mesh points using PCA projection + RANSAC.
    Returns line origin and 3D direction unit vector.
    """
    # Step 1: PCA to find the best-fit plane
    pca = PCA(n_components=2)
    projected = pca.fit_transform(points)

    # Step 2: Fit a line to the 2D projection using RANSAC
    ransac = RANSACRegressor().fit(projected[:, 0].reshape(-1, 1), projected[:, 1])
    slope = ransac.estimator_.coef_[0]

    # Line direction in 2D PCA space
    line_2d = np.array([1.0, slope])
    line_2d /= np.linalg.norm(line_2d)

    # Map line_2d to 3D using PCA components
    # line_dir_3d = a * pca.components_[0] + b * pca.components_[1]
    a, b = line_2d
    dir_3d = a * pca.components_[0] + b * pca.components_[1]
    dir_3d /= np.linalg.norm(dir_3d)

    return pca.mean_, dir_3d



In [22]:
# RANSAC

def align_mesh_to_direction(mesh, center, direction):
    """
    Rotate and translate the mesh so the given direction aligns with global X.
    """
    x_axis = direction
    up_hint = np.array([0, 0, 1])  # try to preserve Z up
    y_axis = up_hint - np.dot(up_hint, x_axis) * x_axis
    y_axis /= np.linalg.norm(y_axis)
    z_axis = np.cross(x_axis, y_axis)

    rotation_matrix = np.column_stack([x_axis, y_axis, z_axis])

    # Center and apply rotation
    centered = mesh.points - center
    rotated = np.dot(centered, rotation_matrix)
    aligned_mesh = mesh.copy()
    aligned_mesh.points = rotated
    return aligned_mesh


In [23]:
# RANSAC

points = mesh.points
center, direction = fit_ransac_line(points)
aligned_mesh = align_mesh_to_direction(mesh, center, direction)

# Check bounds
bounds = aligned_mesh.bounds
print("Aligned bounding box:")
print(f"X: {bounds[0]:.2f} to {bounds[1]:.2f} ({bounds[1] - bounds[0]:.2f})")

# Preview
plotter = pv.Plotter()
plotter.add_mesh(aligned_mesh, show_edges=True, color='lightgray')
plotter.add_mesh(aligned_mesh.bounding_box(), style='wireframe', color='red', line_width=2)
plotter.add_axes()
plotter.screenshot("ransac_aligned_mesh.png")


Aligned bounding box:
X: -1066.35 to 1216.74 (2283.09)


pyvista_ndarray([[[255, 255, 255],
                  [255, 255, 255],
                  [255, 255, 255],
                  ...,
                  [255, 255, 255],
                  [255, 255, 255],
                  [255, 255, 255]],

                 [[255, 255, 255],
                  [255, 255, 255],
                  [255, 255, 255],
                  ...,
                  [255, 255, 255],
                  [255, 255, 255],
                  [255, 255, 255]],

                 [[255, 255, 255],
                  [255, 255, 255],
                  [255, 255, 255],
                  ...,
                  [255, 255, 255],
                  [255, 255, 255],
                  [255, 255, 255]],

                 ...,

                 [[255, 255, 255],
                  [255, 255, 255],
                  [255, 255, 255],
                  ...,
                  [255, 255, 255],
                  [255, 255, 255],
                  [255, 255, 255]],

                 [[255, 255, 255],
  

Minimum Volume Bounding Box (MBBox) via Rotation Search

In [24]:
import pyvista as pv
import numpy as np

def find_best_rotation(mesh, axis='z', steps=180):
    """
    Rotate the mesh in 2D (around Z) and find the rotation
    with the minimum bounding box volume.
    """
    angles = np.linspace(0, 180, steps)
    best_volume = np.inf
    best_angle = 0
    best_rotated = None

    for angle in angles:
        rotated = mesh.copy()
        rotated.rotate_z(angle, inplace=True)

        bounds = rotated.bounds
        dx = bounds[1] - bounds[0]
        dy = bounds[3] - bounds[2]
        dz = bounds[5] - bounds[4]
        volume = dx * dy * dz

        if volume < best_volume:
            best_volume = volume
            best_angle = angle
            best_rotated = rotated

    return best_rotated, best_angle, best_volume


In [25]:
best_mesh, best_angle, best_vol = find_best_rotation(mesh, steps=360)
print(f"Best rotation angle (Z): {best_angle:.2f}°")
print(f"Volume of bounding box: {best_vol:.2f}")

# Show result
bounds = best_mesh.bounds
print("Aligned bounding box:")
print(f"X: {bounds[0]:.2f} to {bounds[1]:.2f} ({bounds[1] - bounds[0]:.2f})")

plotter = pv.Plotter()
plotter.add_mesh(best_mesh, show_edges=True, color='lightgray')
plotter.add_mesh(best_mesh.bounding_box(), style='wireframe', color='red', line_width=2)
plotter.add_axes()
plotter.screenshot("mbbox_aligned_mesh.png")


Best rotation angle (Z): 0.00°
Volume of bounding box: 94327065.60
Aligned bounding box:
X: -101.80 to 101.80 (203.60)


pyvista_ndarray([[[255, 255, 255],
                  [255, 255, 255],
                  [255, 255, 255],
                  ...,
                  [255, 255, 255],
                  [255, 255, 255],
                  [255, 255, 255]],

                 [[255, 255, 255],
                  [255, 255, 255],
                  [255, 255, 255],
                  ...,
                  [255, 255, 255],
                  [255, 255, 255],
                  [255, 255, 255]],

                 [[255, 255, 255],
                  [255, 255, 255],
                  [255, 255, 255],
                  ...,
                  [255, 255, 255],
                  [255, 255, 255],
                  [255, 255, 255]],

                 ...,

                 [[255, 255, 255],
                  [255, 255, 255],
                  [255, 255, 255],
                  ...,
                  [255, 255, 255],
                  [255, 255, 255],
                  [255, 255, 255]],

                 [[255, 255, 255],
  

3D Bounding box using rotation

In [26]:
import pyvista as pv
import numpy as np
import itertools

def find_best_3d_rotation(mesh, angle_step=10):
    angles = np.arange(0, 180 + angle_step, angle_step)
    best_volume = np.inf
    best_rotation = (0, 0, 0)
    best_mesh = None

    for rx, ry, rz in itertools.product(angles, repeat=3):
        rotated = mesh.copy()
        rotated.rotate_x(rx, inplace=True)
        rotated.rotate_y(ry, inplace=True)
        rotated.rotate_z(rz, inplace=True)

        bounds = rotated.bounds
        dx = bounds[1] - bounds[0]
        dy = bounds[3] - bounds[2]
        dz = bounds[5] - bounds[4]
        volume = dx * dy * dz

        if volume < best_volume:
            best_volume = volume
            best_rotation = (rx, ry, rz)
            best_mesh = rotated

    return best_mesh, best_rotation, best_volume


In [27]:
best_mesh, best_angles, best_vol = find_best_3d_rotation(mesh, angle_step=10)
print("Best rotation (X, Y, Z):", best_angles)
print("Bounding box volume:", best_vol)

plotter = pv.Plotter()
plotter.add_mesh(best_mesh, show_edges=True, color='lightgray')
plotter.add_mesh(best_mesh.bounding_box(), style='wireframe', color='red', line_width=2)
plotter.add_axes()
plotter.screenshot("mbbox_3d_result.png")


Best rotation (X, Y, Z): (0, 0, 0)
Bounding box volume: 94327065.6


pyvista_ndarray([[[255, 255, 255],
                  [255, 255, 255],
                  [255, 255, 255],
                  ...,
                  [255, 255, 255],
                  [255, 255, 255],
                  [255, 255, 255]],

                 [[255, 255, 255],
                  [255, 255, 255],
                  [255, 255, 255],
                  ...,
                  [255, 255, 255],
                  [255, 255, 255],
                  [255, 255, 255]],

                 [[255, 255, 255],
                  [255, 255, 255],
                  [255, 255, 255],
                  ...,
                  [255, 255, 255],
                  [255, 255, 255],
                  [255, 255, 255]],

                 ...,

                 [[255, 255, 255],
                  [255, 255, 255],
                  [255, 255, 255],
                  ...,
                  [255, 255, 255],
                  [255, 255, 255],
                  [255, 255, 255]],

                 [[255, 255, 255],
  

In [29]:
xmin, xmax, ymin, ymax, zmin, zmax = best_mesh.bounds

length = xmax - xmin  # beam length (now aligned with X)
height = ymax - ymin  # beam height (Y)
width  = zmax - zmin  # flange width (Z)
center = best_mesh.center

print("📏 Aligned Beam Dimensions:")
print(f"  Length (X): {length:.2f} mm")
print(f"  Height (Y): {height:.2f} mm")
print(f"  Width  (Z): {width:.2f} mm")
print(f"  Beam center (X, Y, Z): {center}")

📏 Aligned Beam Dimensions:
  Length (X): 203.60 mm
  Height (Y): 203.20 mm
  Width  (Z): 2280.00 mm
  Beam center (X, Y, Z): (0.0, 7.105427357601002e-15, 1140.0)


In [30]:
def shift_origin_to_corner(mesh):
    """
    Translate mesh so its bounding box min corner becomes the origin (0, 0, 0).
    """
    xmin, xmax, ymin, ymax, zmin, zmax = mesh.bounds
    translation = np.array([xmin, ymin, zmin])
    
    shifted = mesh.copy()
    shifted.points -= translation
    return shifted, translation

In [31]:
aligned_mesh_origin, origin_translation = shift_origin_to_corner(best_mesh)

print("📐 Origin shifted to (0,0,0)")
print("Translation applied:", origin_translation)

bounds = aligned_mesh_origin.bounds
print("New dimensions:")
print(f"  Length (X): {bounds[1] - bounds[0]:.2f}")
print(f"  Height (Y): {bounds[3] - bounds[2]:.2f}")
print(f"  Width  (Z): {bounds[5] - bounds[4]:.2f}")

📐 Origin shifted to (0,0,0)
Translation applied: [-101.8 -101.6    0. ]
New dimensions:
  Length (X): 203.60
  Height (Y): 203.20
  Width  (Z): 2280.00


get holes from geometry

In [36]:
from scipy.spatial.transform import Rotation as R

def rotate_point_3d(point, angles_deg):
    """Rotate a point by rx, ry, rz degrees."""
    r = R.from_euler('xyz', angles_deg, degrees=True)
    return r.apply(point)


In [37]:
def transform_holes_with_angles(holes, angles_deg, translation):
    transformed = []
    for h in holes:
        center = h["center"]
        # Rotate
        rotated = rotate_point_3d(center, angles_deg)
        # Translate (shift origin)
        shifted = rotated - translation
        transformed.append({**h, "transformed_center": shifted})
    return transformed


In [38]:
def plot_holes_on_mesh(mesh, transformed_holes, sphere_radius=5.0):
    plotter = pv.Plotter()
    plotter.add_mesh(mesh, color='lightgray', show_edges=True)

    for h in transformed_holes:
        sphere = pv.Sphere(radius=sphere_radius, center=h["transformed_center"])
        plotter.add_mesh(sphere, color='red')

    plotter.add_axes()
    plotter.screenshot("holes_on_mesh.png")
    print("✅ Hole positions visualized in holes_on_mesh.png")


In [39]:
# 1. Extract holes
holes = extract_cylindrical_holes(shape)

# 2. Apply 3D rotation and corner shift
transformed_holes = transform_holes_with_angles(holes, best_angles, origin_translation)

# 3. Visualize
plot_holes_on_mesh(aligned_mesh_origin, transformed_holes)


✅ Hole positions visualized in holes_on_mesh.png


Generate Hole Table

In [42]:
import pandas as pd

def make_hole_table(transformed_holes):
    rows = []
    for i, h in enumerate(transformed_holes):
        x, y, z = h["transformed_center"]
        rows.append({
            "Hole ID": i + 1,
            "Diameter (mm)": round(h["radius"], 2)*2,
            "X (mm)": round(x, 2),
            "Y (mm)": round(y, 2),
            "Z (mm)": round(z, 2)
        })
    return pd.DataFrame(rows)


In [44]:
df_holes = make_hole_table(transformed_holes)
display(df_holes)  # If in Jupyter
# print(df_holes)
# df_holes.to_csv("hole_positions.csv", index=False)

,Hole ID,Diameter (mm),X (mm),Y (mm),Z (mm)
0,1,12.0,-2192.67,51.60,2000.0
1,2,12.0,-2192.67,51.60,1500.0
2,3,12.0,-2192.67,51.60,900.0
3,4,12.0,-2192.67,51.60,300.0
4,5,12.0,-2192.67,151.60,300.0
5,6,12.0,-2192.67,151.60,900.0
6,7,12.0,-2192.67,151.60,1500.0
7,8,12.0,-2192.67,151.60,2000.0
8,9,12.0,50.00,2298.07,425.0
9,10,12.0,50.00,2298.07,425.0


Generate holes DSTV for checking, this includes all faces.

In [50]:
def classify_face(hole_direction, tolerance=0.9):
    axis_map = {
        "BO": np.array([0, 0, 1]),
        "SI": np.array([0, 0, -1]),
        "KO": np.array([0, 1, 0]),
        "UN": np.array([0, -1, 0]),
        "RI": np.array([1, 0, 0]),
        "LI": np.array([-1, 0, 0]),
    }

    for face, normal in axis_map.items():
        dot = np.dot(hole_direction / np.linalg.norm(hole_direction), normal)
        if dot > tolerance:
            return face
    return "UNKNOWN"


In [55]:
# Extract dimensions from aligned mesh
xmin, xmax, ymin, ymax, zmin, zmax = aligned_mesh_origin.bounds

beam_length = xmax - xmin  # global X
beam_height = ymax - ymin  # global Y
web_thickness = zmax - zmin  # global Z
beam_size = "UC203x203x46"  # or whatever your section is


In [56]:
def write_full_dstv_nc1(filename, holes, beam_length, beam_height, material="S355JR", thickness=None, beam_size=None):
    from collections import defaultdict
    grouped = defaultdict(list)

    # Group holes by face
    for h in holes:
        face = classify_face(h["direction"])
        if face == "UNKNOWN":
            continue
        grouped[face].append(h)

    with open(filename, "w") as f:
        # Header
        f.write("ST\n")
        if beam_size:
            f.write(f"  NAME {filename} / {beam_size}\n")
        else:
            f.write(f"  NAME {filename}\n")
        f.write(f"  MATERI {material}\n")
        f.write("EN\n")

        # Main geometry: AK block
        f.write("AK\n")
        f.write(f"  L {beam_length:.2f}\n")
        f.write(f"  B {beam_height:.2f}\n")
        if thickness:
            f.write(f"  D {thickness:.2f}\n")
        f.write("EN\n")

        # Drilling faces
        for face in ["BO", "SI", "KO", "UN", "RI", "LI"]:
            if face in grouped:
                f.write(f"{face}\n")
                for h in grouped[face]:
                    x, y, z = h["transformed_center"]
                    dia = h["radius"] * 2
                    if face in ["BO", "SI"]:
                        f.write(f"  {x:.2f} {y:.2f} {dia:.2f}\n")
                    elif face in ["KO", "UN"]:
                        f.write(f"  {x:.2f} {z:.2f} {dia:.2f}\n")
                    elif face in ["RI", "LI"]:
                        f.write(f"  {y:.2f} {z:.2f} {dia:.2f}\n")
                f.write("EN\n")

        # End of file
        f.write("EN\n")


In [69]:
write_full_dstv_nc1(
    filename="beam_full.nc1",
    holes=transformed_holes,
    beam_length=beam_length,
    beam_height=beam_height,
    thickness=web_thickness,
    material="S355JR",
    beam_size=beam_size
)
print("✅ Full DSTV file written: beam_full.nc1")


✅ Full DSTV file written: beam_full.nc1
